In [11]:
import json

In [30]:
import inspect

In [35]:
from typing import Dict, Any

In [41]:
def that():
    return
def this(x, y=2, z:str=3, w:Dict[str,Any]=10):
    return 

In [39]:
inspect.getfullargspec(this).annotations

{'z': str, 'w': typing.Dict[str, typing.Any]}

In [42]:
inspect.getfullargspec(that).annotations

{}

In [40]:
inspect.getfullargspec(this).get("annotations")

AttributeError: 'FullArgSpec' object has no attribute 'get'

In [29]:
import yaml
print(yaml.dump({"a":1,"b":{"yes":True,"No":False}, "list":[1,2,3,{"oh":"no", "not":"this"}]}))

a: 1
b:
  'No': false
  'yes': true
list:
- 1
- 2
- 3
- not: this
  oh: 'no'



In [15]:
from apispec import APISpec
from apispec_webframeworks.flask import FlaskPlugin
import flask
from flask import Flask
from typing import List, Callable
from . import parse_args

def create_apispec(appname:str, version:str = "1.0.0") -> APISpec:
    # Create an APISpec
    spec = APISpec(
        title=appname,
        version=version,
        openapi_version="3.0.2",
        plugins=[FlaskPlugin()],
    )
    return spec

def jigger_the_docstring(function: Callable):
    docstring = function.__doc__
    
    if "---" in docstring:
        return
    
    openapi_docstring = f"""
    get:
        summary: GET function.__name__
        description: {docstring}
        {swagger_query_params(function)}
        {swagger_positional_args(function)}
    """

def swagger_format(in, name, description, type=None, default=None, **other_stuff):
    params = {
        "parameters":
        [{
            "in":in,
            "name":name,
            "description":description,
            "schema":{
                "type":type,
                "default":default
            },
            **other_stuff
        }]
    return params
    
def swagger_it(function: Callable):
    positional, named = parse_args(function)
    in = "path"
    
    return swagger_format(in=in, name=Callable.__name__, description=function.__doc__)
        
def register_function_specs(app: flask.Flask, spec: APISpec):
    # Register the path and the entities within it
    view_functions = app.view_functions
    with app.test_request_context():
        for function in view_functions.values():
            spec.path(view=function)



# Optional Flask support
app = Flask(__name__)


@app.route("/random")
def random_animal():
    """ Cool Foo-Bar route.
    ---
    get:
        summary: Foo-Bar endpoint.
        description: Get a single foo with the bar ID.
        parameters:
            - name: bar_id
              in: path
              description: Bar ID
              schema:
                  type: python Dict[str,Any]
                  required: true
            - name: aggFunc
              in: query
              description: The funcs arfe gging
              schema:
                  type: string
              default: empty
            - name: test
              schema:
                  type: custom
              in: query
              description: this is just a test
        responses:
            200:
                description: Foo object to be returned.
                schema: FooSchema
            404:
                description: Foo not found.
    """
    # Hardcoded example data
    pet_data = {
        "name": "sample_pet_" + str(uuid.uuid1()),
        "categories": [{"id": 1, "name": "sample_category"}],
    }
    return pet_data


@app.route("/person")
def random_person():
    """ Cool Foo-Bar route.
    ---
    get:
        summary: Foo-Bar endpoint.
        description: Get a single foo with the bar ID.
        parameters:
            - in: query
              name: offset
              schema:
                type: integer
              description: The number of items to skip before starting to collect the result set
            - in: query
              name: limit
              schema:
                type: integer
              description: The numbers of items to return
    """
    return {"message":"this"}


In [24]:
func_name="random_pet"
app.view_functions[func_name]

<function __main__.<lambda>(x)>

In [8]:
swagger_ui = """<!DOCTYPE html>
<html>
  <head>
    <title>Swagger</title>
    <meta charset="utf-8"/>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" type="text/css" href="//unpkg.com/swagger-ui-dist@3/swagger-ui.css" />
  </head>
  <body>
    <div id="swagger-ui"></div>
    <script src="//unpkg.com/swagger-ui-dist@3/swagger-ui-bundle.js"></script>
    <script>
    const ui = SwaggerUIBundle({
        url: "/swagger",
        dom_id: '#swagger-ui',
        presets: [
          SwaggerUIBundle.presets.apis,
          SwaggerUIBundle.SwaggerUIStandalonePreset
        ],
        layout: "BaseLayout",
        requestInterceptor: (request) => {
          request.headers['X-CSRFToken'] = "{{ csrf_token }}"
          return request;
        }
      })
    </script>
  </body>
</html>"""

@app.route("/")
def read_swagger():
    return swagger_ui


import json
@app.route("/swagger")
def serve_swagger():
    return json.dumps(spec.to_dict())

In [9]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Mar/2020 21:35:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2020 21:35:11] "GET /swagger HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2020 21:35:11] "GET /favicon.ico HTTP/1.1" 404 -
